<a href="https://colab.research.google.com/github/Yassmina-Abdo/pyspark_course/blob/main/patients_classification_mlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 2 PySpark:

### Machine Learning

#### But first, let's do some SQL :)

**First, install and import PySPark and SparkSession**

In [1]:
!pip install pyspark --q

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 44 kB/s 
     |████████████████████████████████| 199 kB 13.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=02a9897dd0e8d42837f81528b14c4c106eac53e9444628942b4cee6b66030bcb
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
import pyspark
from pyspark.sql import SparkSession

In [39]:
from pyspark.sql.functions import when, lit

**Now, download the dataset**

In [3]:
!gdown https://drive.google.com/uc?id=1PB6wBDVTM_eocxOyi0lWlLBQOlH0rLe_ -O PatientInfo.csv

Downloading...
From: https://drive.google.com/uc?id=1PB6wBDVTM_eocxOyi0lWlLBQOlH0rLe_
To: /content/PatientInfo.csv
100% 489k/489k [00:00<00:00, 66.5MB/s]


**Create a SparkSession object and name the app "Lab2"**

In [104]:
spark = SparkSession.builder.master("local[1]").appName('Lab2').getOrCreate()

**1. Read the file PatientInfo.csv into a dataframe**

In [105]:
df = spark.read.csv('/content/PatientInfo.csv', header = True, inferSchema=True)

**Show the first 5 lines of the dataframe**

In [106]:
df.show(5)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|     confirmed_date|      released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|2020-01-23 00:00:00|2020-02-05 00:00:00|         null|released|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|2020-01-30 00:00:00|2020-03-02 00:00:00|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|

**Now do the same but using SQL select statement**

1. Create a temporary view (table) called patients

In [107]:
df.createOrReplaceTempView("patients")

2. Use SELECT statement to select all columns from the dataframe

In [108]:
spark.sql("SELECT *  FROM patients").show()

+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------+--------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|     confirmed_date|      released_date|deceased_date|   state|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|2020-01-23 00:00:00|2020-02-05 00:00:00|         null|released|
|1000000002|  male|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       null|            31|              null|2020-01-30 00:00:00|2020-03-02 00:00:00|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|   Jongno-gu|contact with patient| 20020

3. Limit the output to only 5 rows *using SQL commands*

In [109]:
spark.sql("SELECT *  FROM patients limit 5").show()

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|     confirmed_date|      released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|2020-01-23 00:00:00|2020-02-05 00:00:00|         null|released|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|2020-01-30 00:00:00|2020-03-02 00:00:00|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|

4. Select the count of males and females in the dataset

In [110]:
spark.sql("select SUM(case when sex = 'male' then 1 end) as Males_count\
                , SUM(case when sex = 'female' then 1 end) as Females_count\
                  FROM patients").show()

+-----------+-------------+
|Males_count|Females_count|
+-----------+-------------+
|       1825|         2218|
+-----------+-------------+



5. Select the count of males and females *as percentage* (how many percent of the data are males and how many are females?)

In [111]:
spark.sql("select 100 * SUM(case when sex = 'male' then 1 end)/count(*) as Males_count\
                , 100 * SUM(case when sex = 'female' then 1 end)/count(*) as Females_count\
                  FROM patients").show()

+-----------------+------------------+
|      Males_count|     Females_count|
+-----------------+------------------+
|35.33397870280736|42.942884801548885|
+-----------------+------------------+



6. How many people did survive, and how many didn't?

In [112]:
spark.sql("select SUM(case when state = 'deceased' then  1 end) as survive\
                , SUM(case when state != 'deceased' then 1 end) as not_survive\
                  FROM patients").show()

+-------+-----------+
|survive|not_survive|
+-------+-----------+
|     78|       5087|
+-------+-----------+



Now, let's perform some preprocessing using SQL:

1. Convert *age* column to double after removing the 's' at the end -- *hint: check SUBSTRING method*
2. Select only the following columns: `['sex', 'age', 'province', 'state']`
3. Store the result of the query in a new dataframe

In [113]:
#1. 
spark.sql("select double(SUBSTRING(age,1,Length(age)-1)) AS Age from patients ").show()

+----+
| Age|
+----+
|50.0|
|30.0|
|50.0|
|20.0|
|20.0|
|50.0|
|20.0|
|20.0|
|30.0|
|60.0|
|50.0|
|20.0|
|80.0|
|60.0|
|70.0|
|70.0|
|70.0|
|20.0|
|70.0|
|70.0|
+----+
only showing top 20 rows



In [114]:
#2., 3.
selected_df= spark.sql("select sex,double(SUBSTRING(age,1,Length(age)-1)) AS Age,province,state from patients ")

Now view the new dataframe to make sure everything is alright

In [115]:
selected_df.show()

+------+----+--------+--------+
|   sex| Age|province|   state|
+------+----+--------+--------+
|  male|50.0|   Seoul|released|
|  male|30.0|   Seoul|released|
|  male|50.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|female|20.0|   Seoul|released|
|female|50.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|  male|30.0|   Seoul|released|
|female|60.0|   Seoul|released|
|female|50.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|  male|80.0|   Seoul|deceased|
|female|60.0|   Seoul|released|
|  male|70.0|   Seoul|released|
|  male|70.0|   Seoul|released|
|  male|70.0|   Seoul|released|
|  male|20.0|   Seoul|released|
|female|70.0|   Seoul|released|
|female|70.0|   Seoul|released|
+------+----+--------+--------+
only showing top 20 rows



**Now, let's get back to spark operations**

Please copy the following operations from your solution in Lab 1
___

Add a **is_dead** column if patient state is not released then it should yield true, else then False

In [116]:
selected_df = selected_df.withColumn("is_dead",\
                  when((selected_df.state == 'deceased'), lit(1))\
                 .otherwise(lit(0)))
selected_df.show()                

+------+----+--------+--------+-------+
|   sex| Age|province|   state|is_dead|
+------+----+--------+--------+-------+
|  male|50.0|   Seoul|released|      0|
|  male|30.0|   Seoul|released|      0|
|  male|50.0|   Seoul|released|      0|
|  male|20.0|   Seoul|released|      0|
|female|20.0|   Seoul|released|      0|
|female|50.0|   Seoul|released|      0|
|  male|20.0|   Seoul|released|      0|
|  male|20.0|   Seoul|released|      0|
|  male|30.0|   Seoul|released|      0|
|female|60.0|   Seoul|released|      0|
|female|50.0|   Seoul|released|      0|
|  male|20.0|   Seoul|released|      0|
|  male|80.0|   Seoul|deceased|      1|
|female|60.0|   Seoul|released|      0|
|  male|70.0|   Seoul|released|      0|
|  male|70.0|   Seoul|released|      0|
|  male|70.0|   Seoul|released|      0|
|  male|20.0|   Seoul|released|      0|
|female|70.0|   Seoul|released|      0|
|female|70.0|   Seoul|released|      0|
+------+----+--------+--------+-------+
only showing top 20 rows



**Please split the data into train and test dataframes**

*Ratio: 80:20 - Seed=42*

In [117]:
trainDF, testDF = selected_df.randomSplit([0.8, 0.2], seed=42)

**Now, let's import RandomForestClassifier and start our ML pipeline**

In [118]:
from pyspark.ml.feature import Imputer
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline

**Create a pipeline that contains the following stages:**

- Imputer: impute the null values in `age` column to the mean value
- StringIndexer: convert `sex` to `is_male` and `province` to `province_index` as numerical values
- OneHotEncoder: perform one hot encoding on both `is_male` and -province_index`
- VectorAssembler: assemble feature vector from the following columns: `'age', 'is_male', 'province_index'`
- RandomForestClassifier: final estimator

In [132]:
#1.
imputer= Imputer(inputCol='Age', outputCol= 'Age').setStrategy("mean")
#2. 
stringindexer= StringIndexer(inputCols=["sex","province"], outputCols=["is_male","province_index"],handleInvalid = "skip")
#3.
onehotEncoder= OneHotEncoder(inputCols=["is_male","province_index"], outputCols=["Is_male","Province_Index"])
#4.
vectorAssembler= VectorAssembler(inputCols = ["Age", "Is_male", "Province_Index"], outputCol = "features",handleInvalid = "skip")
#5.
model= RandomForestClassifier(featuresCol='features',labelCol='is_dead')

In [133]:
pipeline = Pipeline(stages=[imputer, stringindexer, onehotEncoder, vectorAssembler,model])

Fit the pipeline to the train dataframe

In [134]:
pl_Model=pipeline.fit(trainDF)

Now transform the test DF to get predictions

In [135]:
predDF = pl_Model.transform(testDF)

Show the final predictions DF

In [136]:
predDF.select('features','is_dead','prediction').show(5,truncate=False)

+---------------------------------------+-------+----------+
|features                               |is_dead|prediction|
+---------------------------------------+-------+----------+
|(18,[0,1,3],[40.0494233937397,1.0,1.0])|0      |0.0       |
|(18,[0,1,3],[40.0494233937397,1.0,1.0])|0      |0.0       |
|(18,[0,1,5],[40.0494233937397,1.0,1.0])|0      |0.0       |
|(18,[0,1,5],[40.0494233937397,1.0,1.0])|0      |0.0       |
|(18,[0,1,5],[40.0494233937397,1.0,1.0])|0      |0.0       |
+---------------------------------------+-------+----------+
only showing top 5 rows



**Model Evaluation**

Now let's evaluate our model! Let's get the accuracy of our model

In [147]:
from pyspark.ml.evaluation import *

**Accuracy**

In [149]:
eval= MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="is_dead")
acc_res  = eval.evaluate(predDF, {eval.metricName:"accuracy"})
print("Accuracy = %0.2f" % acc_res)

Accuracy = 0.98


**ROC**

In [150]:
eval= BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='is_dead',metricName='areaUnderROC')
print(f'Roc= ',eval.evaluate(predDF))

Roc=  0.5


Excellent! Now let's generate the confusion matrix of our predictions

*Hint: we can use `scikit-learn`'s `classification_report`. You will need to transform the predictions into pandas DF first*

In [145]:
from sklearn.metrics import classification_report

In [146]:
predDF_pd = predDF.toPandas()
print(classification_report(predDF_pd['is_dead'], predDF_pd['prediction']))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       790
           1       0.00      0.00      0.00        20

    accuracy                           0.98       810
   macro avg       0.49      0.50      0.49       810
weighted avg       0.95      0.98      0.96       810



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Great Job!

**If you followed the instructions correctly, you should get a total accuracy of 89%, and F1 scores of 92% and 85%**

**Do you think you can improve this accuracy? Let's see what you can do :)**

___
If you have any questions you can reach out to me:

### Omar Hammad
#### Software Engineer
##### Email: ommar365@gmail.com
##### Phone: 01144070145
##### Linkedin: https://www.linkedin.com/in/omar-a-hammad